In [2]:
import pandas as pd
from os import path
import sqlite3

# Directory where data is stored
DATA_DIR = '../resources/code-soccer-files-main/data'

# create connection
conn = sqlite3.connect(path.join(DATA_DIR, 'soccer-data.sqlite'))

# load csv data
player_match = pd.read_csv(path.join(DATA_DIR, 'player_match.csv'))
player = pd.read_csv(path.join(DATA_DIR, 'players.csv'))
game = pd.read_csv(path.join(DATA_DIR, 'matches.csv'))
team = pd.read_csv(path.join(DATA_DIR, 'teams.csv'))

In [3]:
# and write it to sql
player_match.to_sql('player_match', conn, index=False, if_exists='replace')
player.to_sql('player', conn, index=False, if_exists='replace')
game.to_sql('game', conn, index=False, if_exists='replace')
team.to_sql('team', conn, index=False, if_exists='replace')

32

Queries

In [4]:
# return entire player table
df = pd.read_sql(
    """
    SELECT *
    FROM player
    """, conn)
df.head()

,player_id,player_name,pos,foot,birth_date,born,weight,height,passport,team_id,team
0,32793,A. N'Diaye,MID,right,19900306,250,82,187,686,19314,Senegal
1,36,T. Alderweireld,DEF,right,19890302,56,91,187,56,5629,Belgium
2,48,J. Vertonghen,DEF,left,19870424,56,88,189,56,5629,Belgium
3,54,C. Eriksen,MID,right,19920214,208,76,180,208,7712,Denmark
4,93,J. Guðmunds­son,MID,left,19901027,352,77,186,352,7839,Iceland


In [5]:
# return specific columns from player table and rename on the fly
df = pd.read_sql(
    """
    SELECT player_id, player_name AS name, team, pos, foot
    FROM player
    """, conn)
df.head()

,player_id,name,team,pos,foot
0,32793,A. N'Diaye,Senegal,MID,right
1,36,T. Alderweireld,Belgium,DEF,right
2,48,J. Vertonghen,Belgium,DEF,left
3,54,C. Eriksen,Denmark,MID,right
4,93,J. Guðmunds­son,Iceland,MID,left


Filtering

In [6]:
# basic WHERE filter, only rows with selected value
df = pd.read_sql(
    """
    SELECT player_id, player_name AS name, pos, foot
    FROM player
    WHERE team = 'Japan'
    """, conn)
df.head()

,player_id,name,pos,foot
0,703,M. Yoshida,DEF,right
1,101592,K. Honda,MID,left
2,37896,E. Kawashima,GKP,right
3,95010,H. Yamaguchi,MID,right
4,14730,T. Usami,FWD,right


In [7]:
# AND in filter
df = pd.read_sql(
    """
    SELECT player_id, player_name AS name, team, pos, foot
    FROM player
    WHERE team = 'Japan' AND pos == 'MID'
    """, conn)
df.head()

,player_id,name,team,pos,foot
0,101592,K. Honda,Japan,MID,left
1,95010,H. Yamaguchi,Japan,MID,right
2,14816,S. Kagawa,Japan,MID,right
3,14836,T. Inui,Japan,MID,right
4,14929,M. Hasebe,Japan,MID,right


In [8]:
# OR in filter
df = pd.read_sql(
    """
    SELECT player_id, player_name AS name, team, pos, foot
    FROM player
    WHERE team = 'Japan' OR pos == 'GKP'
    """, conn)
df.head()

,player_id,name,team,pos,foot
0,703,M. Yoshida,Japan,DEF,right
1,101576,I. Akinfeev,Russia,GKP,right
2,101592,K. Honda,Japan,MID,left
3,3397,Arrizabalaga,Spain,GKP,right
4,3551,W. Caballero,Argentina,GKP,right


In [9]:
# IN in filter
df = pd.read_sql(
    """
    SELECT player_id, player_name AS name, pos, foot
    FROM player
    WHERE pos IN ('DEF', 'MID')
    """, conn)
df.head()

,player_id,name,pos,foot
0,32793,A. N'Diaye,MID,right
1,36,T. Alderweireld,DEF,right
2,48,J. Vertonghen,DEF,left
3,54,C. Eriksen,MID,right
4,93,J. Guðmunds­son,MID,left


In [10]:
# negation with NOT
df = pd.read_sql(
    """
    SELECT player_id, player_name AS name, team, pos, foot
    FROM player
    WHERE team NOT IN ('Japan', 'Iceland')
    """, conn)
df.head()

,player_id,name,team,pos,foot
0,32793,A. N'Diaye,Senegal,MID,right
1,36,T. Alderweireld,Belgium,DEF,right
2,48,J. Vertonghen,Belgium,DEF,left
3,54,C. Eriksen,Denmark,MID,right
4,122,D. Mertens,Belgium,FWD,right


Joining

In [14]:
# no WHERE so fullcrossjoin - every player is matched with every team
df = pd.read_sql(
    """
    SELECT
        player.player_name as name,
        player.pos,
        player.team as player_team,
        team.team as team_team,
        team.grouping
    FROM player, team
    """, conn)
df.head(10)

,name,pos,player_team,team_team,grouping
0,A. N'Diaye,MID,Senegal,Korea Republic,F
1,A. N'Diaye,MID,Senegal,Russia,A
2,A. N'Diaye,MID,Senegal,Sweden,F
3,A. N'Diaye,MID,Senegal,Tunisia,G
4,A. N'Diaye,MID,Senegal,Brazil,E
5,A. N'Diaye,MID,Senegal,Germany,F
6,A. N'Diaye,MID,Senegal,Morocco,B
7,A. N'Diaye,MID,Senegal,Colombia,H
8,A. N'Diaye,MID,Senegal,Panama,G
9,A. N'Diaye,MID,Senegal,Argentina,D


In [15]:
# n of rows
print(df.shape)

# works when we add WHERE to filter after crossjoin
df = pd.read_sql(
    """
    SELECT
        player.player_name as name,
        player.pos,
        player.team,
        team.grouping
    FROM player, team
    WHERE player.team = team.team
    """, conn)
df.head()

(23520, 5)


,name,pos,team,grouping
0,A. N'Diaye,MID,Senegal,H
1,T. Alderweireld,DEF,Belgium,G
2,J. Vertonghen,DEF,Belgium,G
3,C. Eriksen,MID,Denmark,C
4,J. Guðmunds­son,MID,Iceland,D


In [16]:
# adding a third table
df = pd.read_sql(
    """
    SELECT
        player.player_name as name,
        player.pos,
        team.team,
        team.city,
        team.grouping,
        player_match.*
    FROM player, team, player_match
    WHERE
        player.team = team.team AND
        player_match.player_id = player.player_id
    """, conn)
df.head()

,name,pos,team,city,grouping,name,team,min,shot,goal,...,air_duel,air_duel_won,gk_leave_line,gk_save_attempt,throw,corner,pos,side,player_rank,started
0,A. N'Diaye,MID,Senegal,Dakar,H,A. N'Diaye,Senegal,65.0,0,0,...,2,2,0,0,0,0,MID,central,0.0059,1
1,A. N'Diaye,MID,Senegal,Dakar,H,A. N'Diaye,Senegal,87.0,0,0,...,2,1,0,0,0,0,MID,central,0.0035,1
2,T. Alderweireld,DEF,Belgium,Bruxelles (Brussel),G,T. Alderweireld,Belgium,90.0,0,0,...,1,0,0,0,0,0,DEF,right,0.0046,1
3,T. Alderweireld,DEF,Belgium,Bruxelles (Brussel),G,T. Alderweireld,Belgium,90.0,0,0,...,0,0,0,0,0,0,DEF,right,0.0052,1
4,T. Alderweireld,DEF,Belgium,Bruxelles (Brussel),G,T. Alderweireld,Belgium,90.0,0,0,...,1,1,0,0,0,0,DEF,right,0.0032,1


In [17]:
# adding a third table - shorthand
df = pd.read_sql(
    """
    SELECT
        p.player_name as name,
        p.pos,
        t.team,
        t.city,
        t.grouping,
        pm.match_id,
        pm.pass
    FROM player AS p, team AS t, player_match AS pm
    WHERE
        p.team = t.team AND
        pm.player_id = p.player_id
    """, conn)
df.head()

,name,pos,team,city,grouping,match_id,pass
0,A. N'Diaye,MID,Senegal,Dakar,H,2057996,28
1,A. N'Diaye,MID,Senegal,Dakar,H,2057999,16
2,T. Alderweireld,DEF,Belgium,Bruxelles (Brussel),G,2057990,105
3,T. Alderweireld,DEF,Belgium,Bruxelles (Brussel),G,2057992,53
4,T. Alderweireld,DEF,Belgium,Bruxelles (Brussel),G,2058007,77


In [18]:
# adding an additional filter
df = pd.read_sql(
    """
    SELECT
        p.player_name as name,
        p.pos,
        t.team,
        t.city,
        t.grouping,
        pm.match_id,
        pm.pass
    FROM player AS p, team AS t, player_match AS pm
    WHERE
        p.team = t.team AND
        pm.player_id = p.player_id AND
        p.pos == 'FWD'
    """, conn)
df.head()

,name,pos,team,city,grouping,match_id,pass
0,D. Mertens,FWD,Belgium,Bruxelles (Brussel),G,2057990,35
1,D. Mertens,FWD,Belgium,Bruxelles (Brussel),G,2057992,31
2,D. Mertens,FWD,Belgium,Bruxelles (Brussel),G,2057994,4
3,D. Mertens,FWD,Belgium,Bruxelles (Brussel),G,2058007,27
4,D. Mertens,FWD,Belgium,Bruxelles (Brussel),G,2058014,18


Limit / top - for checking results without running the entire query

SELECT *<br>
FROM player<br>
LIMIT 5<br>

SELECT TOP 5 *<br>
FROM player<br>

In [20]:
# including DISTINCT after SELECT drops duplicate observations
df = pd.read_sql(
    """
    SELECT DISTINCT ref, ref2
    FROM game
    """, conn)
df.head()

,ref,ref2
0,378051,378038
1,378204,378144
2,384946,384962
3,380597,380580
4,378232,378231


### Union - stick data on top of each other to form one table

SELECT *<br>
FROM player_data_group_a<br>
UNION <br>
SELECT *<br>
FROM player_data_group_b<br>

### SUBQUERIES - nested SELECT statements

SELECT ... FROM  table_name AS alias -> replace table name with another, inner SELECT ... FROM query and wrap it in parenthesis.


### LEFT, RIGHT, OUTER JOINS

SELECT *<br>
FROM <left_table><br>
LEFT JOIN <right_table><br>
ON <left_table>.<common_column> = <right_table>.<common_column>

In [23]:
df = pd.read_sql(
    """
    SELECT a.*, b.min, b.pass, b.shot, b.goal
    FROM
        (SELECT match_id, label, home as team, away as opp, player_id, player_name
        FROM game, player
        WHERE game.home = player.team_id
        UNION
        SELECT match_id, label, home as team, away as opp, player_id, player_name
        FROM game, player
        WHERE game.away = player.team_id) AS a
    LEFT JOIN player_match AS b ON a.match_id = b.match_id AND a.player_id = b.player_id
    """, conn)

df.head()

,match_id,label,team,opp,player_id,player_name,min,pass,shot,goal
0,2057954,"Russia - Saudi Arabia, 5 - 0",14358,16521,4513,D. Cheryshev,66.0,22.0,3.0,2.0
1,2057954,"Russia - Saudi Arabia, 5 - 0",14358,16521,41123,Mário Fernandes,90.0,26.0,0.0,0.0
2,2057954,"Russia - Saudi Arabia, 5 - 0",14358,16521,101576,I. Akinfeev,NaN,17.0,0.0,0.0
3,2057954,"Russia - Saudi Arabia, 5 - 0",14358,16521,101583,S. Ignashevich,90.0,26.0,0.0,0.0
4,2057954,"Russia - Saudi Arabia, 5 - 0",14358,16521,101590,A. Dzagoev,24.0,8.0,0.0,0.0


In [25]:
# Kudryahsov id = 101647
df.query("player_id == 101647")

# didn't play the first game, but data still in table

,match_id,label,team,opp,player_id,player_name,min,pass,shot,goal
5,2057954,"Russia - Saudi Arabia, 5 - 0",14358,16521,101647,F. Kudryashov,NaN,NaN,NaN,NaN
99,2057956,"Russia - Egypt, 3 - 1",14358,16129,101647,F. Kudryashov,NaN,3.0,0.0,0.0
200,2057958,"Uruguay - Russia, 3 - 0",15670,14358,101647,F. Kudryashov,90.0,38.0,0.0,0.0
2220,2058004,"Spain - Russia, 1 - 1 (P)",1598,14358,101647,F. Kudryashov,120.0,12.0,0.0,0.0
2593,2058012,"Russia - Croatia, 2 - 2 (P)",14358,9598,101647,F. Kudryashov,120.0,38.0,0.0,0.0


In [26]:
df.loc[df['player_name'] == 'L. Messi']

,match_id,label,team,opp,player_id,player_name,min,pass,shot,goal
702,2057972,"Argentina - Iceland, 1 - 1",12274,7839,3359,L. Messi,90.0,67.0,7.0,0.0
797,2057974,"Argentina - Croatia, 0 - 3",12274,9598,3359,L. Messi,90.0,27.0,1.0,0.0
895,2057976,"Nigeria - Argentina, 1 - 2",16823,12274,3359,L. Messi,90.0,59.0,1.0,1.0
2144,2058003,"France - Argentina, 4 - 3",4418,12274,3359,L. Messi,90.0,33.0,3.0,0.0


In [27]:
game = pd.read_sql(
    """
    SELECT *
    FROM game
    """
    , conn)

print(game.loc[[11, 0, 1, 2, 3]])
print()
print(game.query("home == 'GB'"))

    match_id                         label    group                 date  \
11   2058015  Croatia - England, 2 - 1 (E)     None  2018-07-11 18:00:00   
0    2058017       France - Croatia, 4 - 2     None  2018-07-15 15:00:00   
1    2058012   Russia - Croatia, 2 - 2 (P)     None  2018-07-07 18:00:00   
2    2057977      Iceland - Croatia, 1 - 2  Group D  2018-06-26 18:00:00   
3    2057974    Argentina - Croatia, 0 - 3  Group D  2018-06-21 18:00:00   

                           venue        dur  gameweek  round_id   home  away  \
11  Olimpiyskiy stadion Luzhniki  ExtraTime         0   4165366   9598  2413   
0   Olimpiyskiy stadion Luzhniki    Regular         0   4165368   4418  9598   
1      Olimpiyskiy Stadion Fisht  Penalties         0   4165365  14358  9598   
2                   Rostov Arena    Regular         3   4165363   7839  9598   
3        Stadion Nizhny Novgorod    Regular         2   4165363  12274  9598   

    ...  loser     ref    ref2    ref3    ref4  home_score  aw